# Treinamento de Reinforcement Learning

algum bla bla bonito

In [1]:
import gym

## Descobrir ambientes gym disponíveis

In [2]:
from gym import envs
env_names = [spec.id for spec in envs.registry.all()]
for name in sorted(env_names):
    print(name)

Acrobot-v1
Adventure-ram-v0
Adventure-ram-v4
Adventure-ramDeterministic-v0
Adventure-ramDeterministic-v4
Adventure-ramNoFrameskip-v0
Adventure-ramNoFrameskip-v4
Adventure-v0
Adventure-v4
AdventureDeterministic-v0
AdventureDeterministic-v4
AdventureNoFrameskip-v0
AdventureNoFrameskip-v4
AirRaid-ram-v0
AirRaid-ram-v4
AirRaid-ramDeterministic-v0
AirRaid-ramDeterministic-v4
AirRaid-ramNoFrameskip-v0
AirRaid-ramNoFrameskip-v4
AirRaid-v0
AirRaid-v4
AirRaidDeterministic-v0
AirRaidDeterministic-v4
AirRaidNoFrameskip-v0
AirRaidNoFrameskip-v4
Alien-ram-v0
Alien-ram-v4
Alien-ramDeterministic-v0
Alien-ramDeterministic-v4
Alien-ramNoFrameskip-v0
Alien-ramNoFrameskip-v4
Alien-v0
Alien-v4
AlienDeterministic-v0
AlienDeterministic-v4
AlienNoFrameskip-v0
AlienNoFrameskip-v4
Amidar-ram-v0
Amidar-ram-v4
Amidar-ramDeterministic-v0
Amidar-ramDeterministic-v4
Amidar-ramNoFrameskip-v0
Amidar-ramNoFrameskip-v4
Amidar-v0
Amidar-v4
AmidarDeterministic-v0
AmidarDeterministic-v4
AmidarNoFrameskip-v0
AmidarNoFrames

### Explorando o ambiente

In [3]:
env = gym.make('FrozenLake8x8-v0')
env.reset()
for _ in range(2000):
    env.render()
    env.step(env.action_space.sample())
env.close()

FFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Left)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Left)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Up)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
  (Left)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFF

### Verificando os espaços de observações e ações

In [4]:
print(f'Espaço de estados: {env.observation_space}')
print(f'Espaço de ações: {env.action_space}')

Espaço de estados: Discrete(64)
Espaço de ações: Discrete(4)


### Veficando as saídas do ambiente

In [5]:
action = env.action_space.sample()
next_state, reward, done, info = env.step(action)

print(f'Ação: {action}')
print(f'Próxima observação: {next_state}')
print(f'Recompensa: {reward}')
print(f'Finalizado: {done}')
print(f'Finalizado: {info}')

Ação: 1
Próxima observação: 19
Recompensa: 0
Finalizado: True
Finalizado: {'prob': 1.0, 'TimeLimit.truncated': False}


### Hiperparâmetros

In [6]:
from parameters import TrainingParameters, LearningParameters, AgentParameters

EPSILON_MIN = 0.05
MAX_NUM_EPISODES = 50000
STEPS_PER_EPISODE = 64
max_num_steps = MAX_NUM_EPISODES * STEPS_PER_EPISODE
ALPHA = 0.1
GAMMA = 0.999
EPSILON_DECAY = 25 * EPSILON_MIN / max_num_steps

train_params = TrainingParameters(MAX_NUM_EPISODES, STEPS_PER_EPISODE)
learn_params = LearningParameters(ALPHA, GAMMA)
agent_params = AgentParameters(EPSILON_MIN, EPSILON_DECAY, 1)

### Agente

In [7]:
from agent import Agent
from q_table import QTable
import numpy as np

agent = Agent(agent_params, env.action_space.n)
q_table = QTable(env.observation_space.n, env.action_space.n, learn_params)
agent.set_q_table(q_table)

### Funções de treino e teste

In [13]:
def train(agent: Agent, env, params: TrainingParameters):
    best_reward = -float('inf')
    for episode in range(MAX_NUM_EPISODES):
        obs = env.reset()
        done = False
        total_reward = 0.0        
        while not done:
            action = agent.get_action(obs)
            next_obs, reward, done, info = env.step(action)
            agent.q_table.learn(obs, action, reward, next_obs)
            obs = next_obs
            total_reward += reward
        if total_reward > best_reward:
            best_reward = total_reward
        print("Episode#:{} reward:{} best_reward:{} eps:{}".format(episode,
            total_reward, best_reward, agent.epsilon))
    return np.argmax(agent.q_table.table, axis=1)


def test(agent: Agent, env, policy):
    done = False
    obs = env.reset()
    total_reward = 0.0
    while not done:
        action = policy[obs]
        next_obs, reward, done, info = env.step(action)   
        # env.render()     
        obs = next_obs
        total_reward += reward
    return total_reward

### Treinamento

In [9]:
learned_policy = train(agent, env, train_params)

2
Episode#:49697 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49698 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49699 reward:0.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49700 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49701 reward:0.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49702 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49703 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49704 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49705 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49706 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49707 reward:0.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49708 reward:0.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49709 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49710 reward:1.0 best_reward:1.0 eps:0.04999999997112782
Episode#:49711 reward:0.0 best_reward:1.0 eps:0.04999999997112782
Episode#

### Teste

In [14]:
# gym_monitor_path = "./gym_monitor_output"
# env = gym.wrappers.Monitor(env, gym_monitor_path, force=True)
n = 1000
test_rewards = np.zeros(n)
for i in range(n):
    r = test(agent, env, learned_policy)
    test_rewards[i] = r

success_rate = np.average(test_rewards)
print('Taxa de sucesso: ',success_rate)

# env.close()

Taxa de sucesso:  0.875
